In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
DATA_DIR = '/home/HDD6TB/datasets/emotions/ABAW'

In [3]:
import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score,roc_auc_score,average_precision_score
import mord

from collections import defaultdict
import os
import glob
import random
import numpy as np
from tqdm import tqdm
import time
import pickle
import pandas as pd
import random
import csv  
import matplotlib.pyplot as plt

In [4]:
def get_names(id):
    name = ""
    if id>=0 and id<10:
        name = "0000" + str(id)
    elif id>=10 and id<100:
        name = "000" + str(id)
    elif id>=100 and id<1000:
        name = "00" + str(id)
    elif id>=1000 and id<10000:
        name = "0" + str(id)
    else:
        name = str(id)
    return name

In [5]:
compare_filenames=lambda x: int(os.path.splitext(x)[0])

In [6]:
import pathlib
import tensorflow as tf
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout, Flatten, LSTM, Activation, MaxPooling2D
from tensorflow.keras.regularizers import l2 as L2_reg
from tensorflow.keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D, Input, Masking, Conv3D, MaxPooling3D, GlobalMaxPool3D
from tensorflow.keras.optimizers import SGD, Adam
import tensorflow.keras.backend as K 

print(tf.__version__)
from tensorflow.compat.v1.keras.backend import set_session 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
set_session(sess)

2.4.1


# Number of frames

In [40]:
data_dir=os.path.join(DATA_DIR,'cropped')
d=os.path.join(DATA_DIR,'videos')
video2len={}
for filename in os.listdir(d):
    fn, ext = os.path.splitext(os.path.basename(filename))
    vid=os.path.join(d,filename)
    cap = cv2.VideoCapture(vid)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video2len[fn]=total_frames+1 #FIX ME!!! NEED TO ADD 1 to teh number of frames for consistency with challeng's organizer
    
    dn=os.path.join(data_dir,fn)
    last_img=''
    if os.path.exists(dn):
        images=[img_name for img_name in os.listdir(dn) if img_name.lower().endswith('.jpg')]
        last_img=sorted(images, key=compare_filenames)[-1]
    elif os.path.exists(dn+'_left'):
        images=[img_name for img_name in os.listdir(dn+'_left') if img_name.lower().endswith('.jpg')]
        last_img=sorted(images, key=compare_filenames)[-1]
    print(fn,total_frames,last_img)
        

27-60-1280x720 2932 02933.jpg
video39 10678 10671.jpg
77-30-1280x720 5672 05673.jpg
91-30-1920x1080 10921 10922.jpg
360 3509 03473.jpg
136 4487 04488.jpg
197 1869 01786.jpg
291 1830 01831.jpg
16-30-1920x1080 5473 05474.jpg
video2 2064 02055.jpg
89-30-1080x1920 630 00631.jpg
408 2978 02978.jpg
110-30-270x480 9978 09979.jpg
324 5614 05614.jpg
14-30-1920x1080 5310 05311.jpg
129-24-1280x720 6153 06154.jpg
153 3677 03678.jpg
314 2260 02260.jpg
255 1691 01692.jpg
284 4054 04054.jpg
298 2836 02836.jpg
206 4476 04475.jpg
video84 525 00525.jpg
387 2911 02911.jpg
286 2677 02678.jpg
55-25-1280x720 1505 01480.jpg
336 4344 04344.jpg
video52 10679 10671.jpg
60-30-1920x1080 4564 04564.jpg
120 3591 03592.jpg
267 3841 03841.jpg
121-24-1920x1080 10628 10566.jpg
349 4054 04054.jpg
46-30-484x360 2944 02945.jpg
218 3849 03830.jpg
video55 8419 08409.jpg
368 2929 02929.jpg
9-15-1920x1080 47434 47435.jpg
384 3451 03451.jpg
228 2155 02155.jpg
video3 7801 07802.jpg
106-30-720x1280 1404 01405.jpg
64-24-640x360 4

video30 8217 08214.jpg
341 3096 03098.jpg
215 3409 03378.jpg
94-30-1920x1080 2101 02102.jpg
421 3615 03615.jpg
138 5583 05584.jpg
117 5465 05466.jpg
video42 9226 09217.jpg
6-30-1920x1080 7985 07986.jpg
332 4665 04560.jpg
409 3608 03608.jpg
44-25-426x240 300 00301.jpg
154 3362 03363.jpg
407 3817 03817.jpg
41-24-1280x720 6160 06161.jpg
5-60-1920x1080-3 6333 06334.jpg
158 3031 03032.jpg
43-30-406x720 2873 02874.jpg
video94 696 00696.jpg
392 3550 03550.jpg
video19 784 00775.jpg
183 2769 02770.jpg
video6 5324 05325.jpg
video65 4571 04572.jpg
250 2928 02928.jpg
5-60-1920x1080-2 6213 06214.jpg
3-25-1920x1080 6188 06189.jpg
363 3240 03240.jpg
233 4479 04360.jpg
52-30-1280x720 1924 01863.jpg
299 2336 02337.jpg
135 3591 03593.jpg
329 5086 05087.jpg
video37 9186 09116.jpg
video62 5579 05579.jpg
423 5265 05265.jpg
416 6511 06512.jpg
122 4127 04128.jpg
21-24-1920x1080 5230 05231.jpg
369 4438 04437.jpg
video16 12486 12487.jpg
223 2797 02797.jpg
video48 2542 02534.jpg
134-30-1280x720 3875 03875.jpg
1

## Tensorflow

In [44]:
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.models import load_model,Model

In [45]:
def mobilenet_preprocess_input(x,**kwargs):
    x[..., 0] -= 103.939
    x[..., 1] -= 116.779
    x[..., 2] -= 123.68
    return x

preprocessing_function=mobilenet_preprocess_input

In [46]:
#model_name='my_mobilenet_new1_ft_sgd'
model_name='mymobilenet_7_ft_sgd_model'
#model_name='age_gender_tf2_224_deep-03-0.13-0.97'
#model_name='affectnet_orig_my_mobilenet_8_ft_60.575_model'
num_classes=7

base_model=load_model('../models/affectnet_emotions/'+model_name+'.h5')
_,w,h,_=base_model.input.shape
print(w,h)

224 224


In [10]:
feature_extractor_model=Model(base_model.input,[base_model.get_layer('global_pooling').output,base_model.get_layer('feats').output,base_model.output])
feature_extractor_model.summary()
print(feature_extractor_model.output)

[<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'global_pooling')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'feats')>,
 <KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'emotion_preds')>]

In [11]:
weights,bias=base_model.get_layer('emotion_preds').get_weights()
print(weights.shape,bias.shape)

def get_probab(features, logits=False):
    x=np.dot(features,weights)+bias
    if logits:
        return x
    #print(x)
    e_x = np.exp(x - np.max(x,axis=0))
    return e_x / e_x.sum(axis=1)[:,None]

(256, 7) (7,)


In [19]:
#def get_features_scores(data_dir):
data_dir=os.path.join(DATA_DIR,'cropped')
print(data_dir)
img_names=[]
X_global_features,X_scores=[],[]
imgs=[]
for filename in tqdm(os.listdir(data_dir)):
    frames_dir=os.path.join(data_dir,filename)
    for img_name in os.listdir(frames_dir):
        if img_name.lower().endswith('.jpg'):
            img=cv2.imread(os.path.join(frames_dir,img_name))
            if img.size:
                img_names.append(filename+'/'+img_name)
                img=cv2.resize(img,(w,h))
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                imgs.append(img)
                if len(imgs)>=512:        
                    inp = preprocessing_function(np.array(imgs, dtype=np.float32))
                    global_features,feats,scores=feature_extractor_model.predict(inp)
                    #print(global_features.shape,feats.shape,scores.shape)
                    if len(X_scores)==0:
                        X_global_features=global_features
                        #X_global_features=feats
                        X_scores=scores
                    else:
                        X_global_features=np.concatenate((X_global_features,global_features),axis=0)
                        #X_global_features=np.concatenate((X_global_features,feats),axis=0)
                        X_scores=np.concatenate((X_scores,scores),axis=0)
                    imgs=[]

if len(imgs)>0:        
    inp = preprocessing_function(np.array(imgs, dtype=np.float32))
    global_features,feats,scores=feature_extractor_model.predict(inp)
    #print(global_features.shape,feats.shape,scores.shape)
    if len(X_scores)==0:
        X_global_features=global_features
        #X_global_features=feats
        X_scores=scores
    else:
        X_global_features=np.concatenate((X_global_features,global_features),axis=0)
        #X_global_features=np.concatenate((X_global_features,feats),axis=0)
        X_scores=np.concatenate((X_scores,scores),axis=0)

print(len(img_names),X_global_features.shape,X_scores.shape)

  0%|          | 0/564 [00:00<?, ?it/s]

/home/HDD6TB/datasets/emotions/ABAW/faces


100%|██████████| 564/564 [5:04:48<00:00, 32.43s/it]    


2658563 (2658563, 1024) (2658563, 7)


In [30]:
filename2featuresAll={img_name:(global_features,scores) for img_name,global_features,scores in zip(img_names,X_global_features,X_scores)}
print(len(filename2featuresAll))

2658563


## PyTorch

In [7]:
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

print(f"Torch: {torch.__version__}")
device = 'cuda'
use_cuda = torch.cuda.is_available()
print(use_cuda)

Torch: 1.7.1+cu110
True


In [10]:
#num_classes=7
if True:
    #PATH='affectnet_vggface2_enet2_gmp_smooth.pt'
    PATH='enet_b2_8.pt'
    #PATH='enet_b2_7.pt'
    IMG_SIZE=260 #224 #
else:
    #PATH='affectnet_vggface2_enet0.pt'
    #PATH='affectnet_vggface2_enet0_new.pt'
    #PATH='enet_b0_7.pt'
    #PATH='enet_b0_8_best_afew.pt'
    PATH='enet_b0_8_best_vgaf.pt'
    IMG_SIZE=224

test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)
np_transforms = transforms.Compose(
    [
        transforms.ToPILImage(None),
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

print(PATH)
feature_extractor_model = torch.load('../models/affectnet_emotions/'+PATH)

enet_b2_8.pt


In [11]:
if False:
    classifier_weights=feature_extractor_model.classifier[0].weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.classifier[0].bias.cpu().data.numpy()
else:
    classifier_weights=feature_extractor_model.classifier.weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.classifier.bias.cpu().data.numpy()
print(classifier_weights.shape,classifier_weights)
print(classifier_bias.shape,classifier_bias)

(8, 1408) [[-0.08584318 -0.01135117  0.01391132 ...  0.05151795  0.04111349
  -0.03133564]
 [ 0.05045482  0.0124158   0.01706044 ...  0.00231916  0.03906726
   0.01346489]
 [-0.04731339  0.01440803 -0.02087102 ...  0.03216076 -0.00023583
   0.03391237]
 ...
 [-0.0733668  -0.02527069  0.01091044 ...  0.00450477 -0.02584679
   0.02445539]
 [ 0.00541873 -0.01534162 -0.02201885 ...  0.03147097  0.04480692
   0.04357051]
 [ 0.04042469 -0.01425645 -0.00016433 ... -0.03822443 -0.03345024
  -0.02694497]]
(8,) [-0.02906979  0.03788731  0.02125058  0.02583097 -0.04595341 -0.01233763
 -0.01429024 -0.02879   ]


In [12]:
feature_extractor_model.classifier=torch.nn.Identity()
feature_extractor_model=feature_extractor_model.to(device)
feature_extractor_model.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
      (1): DepthwiseSeparableConv(


In [13]:
def get_probab(features, logits=True):
    x=np.dot(features,np.transpose(classifier_weights))+classifier_bias
    if logits:
        return x
    #print(x)
    e_x = np.exp(x - np.max(x,axis=0))
    return e_x / e_x.sum(axis=1)[:,None]

In [24]:
print(test_transforms)
data_dir=os.path.join(DATA_DIR,'cropped')
#data_dir=os.path.join(DATA_DIR,'my_faces/mtcnn_cropped/')
print(data_dir)
img_names=[]
X_global_features=[]
imgs=[]
for filename in tqdm(os.listdir(data_dir)):
#for filename in ['video10_1_left','49-30-1280x720_left', 'video5_left']: #need to copy these directories from cropped_aligned
    frames_dir=os.path.join(data_dir,filename)    
    for img_name in os.listdir(frames_dir):
        if img_name.lower().endswith('.jpg'):
            img = Image.open(os.path.join(frames_dir,img_name))
            img_tensor = test_transforms(img)
            if img.size:
                img_names.append(filename+'/'+img_name)
                imgs.append(img_tensor)
                if len(imgs)>=48: #96: #64: #32:        
                    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features=features.data.cpu().numpy()
                    #print(features.shape)

                    if len(X_global_features)==0:
                        X_global_features=features
                    else:
                        X_global_features=np.concatenate((X_global_features,features),axis=0)
                    imgs=[]

if len(imgs)>0:        
    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
    features=features.data.cpu().numpy()

    if len(X_global_features)==0:
        X_global_features=features
    else:
        X_global_features=np.concatenate((X_global_features,features),axis=0)

    imgs=[]

    #X_scores=X_global_features #get_probab(X_global_features)
    #print(X_global_features.shape,X_scores.shape)

Compose(
    Resize(size=(260, 260), interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
/home/HDD6TB/datasets/emotions/ABAW/faces


In [25]:
X_scores=get_probab(X_global_features)

In [14]:
filename2featuresAll={img_name:(global_features,scores) for img_name,global_features,scores in zip(img_names,X_global_features,X_scores)}
#filename2featuresAll={img_name:(global_features,) for img_name,global_features, in zip(img_names,X_global_features)}
print(len(filename2featuresAll))

2658563


### Save/load features 

In [25]:
import pickle
if False:
    model_name='mymobilenet_7_ft_sgd_model'
    num_classes=7
else:
    num_classes=8
    model_name='enet_b0_8_best_vgaf' #first three challenges
    #model_name='enet2_8' #MTL challenge
    
MODEL2FEATURES=model_name+'.pickle' #'_feat_bgr_emotiw.pickle'

print(MODEL2FEATURES)

enet_b0_8_best_vgaf.pickle


In [40]:
if False:
    with open(MODEL2FEATURES, 'wb') as handle:
        pickle.dump(filename2featuresAll, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
with open(MODEL2FEATURES, 'rb') as handle:
    filename2featuresAll=pickle.load(handle)
print(len(filename2featuresAll))

2658563


# Classifiers

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
#from tensorflow.keras.applications import efficientnet as enet
import efficientnet.tfkeras as enet
#from tensorflow.keras.utils.generic_utils import CustomObjectScope
from tensorflow.keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from tensorflow.keras.metrics import top_k_categorical_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
from copy import deepcopy
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_model_weights = deepcopy(self.model.get_weights())

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_model_weights = deepcopy(self.model.get_weights())

In [11]:
def one_hot_transfer(label,class_num):
    return np.eye(class_num)[label]

def metric_for_Exp(gt,pred,class_num=8):
    # compute_acc
    acc = accuracy_score(gt,pred)
    # compute_F1
    gt = one_hot_transfer(gt,class_num)
    pred = one_hot_transfer(pred,class_num)
    F1 = []
    for i in range(class_num):
        gt_ = gt[:,i]
        pred_ = pred[:,i]
        F1.append(f1_score(gt_.flatten(), pred_))
    F1_mean = np.mean(F1)
    return F1_mean,acc,F1

In [12]:
def CCC_score(x, y):
    vx = x - np.mean(x)
    vy = y - np.mean(y)
    rho = np.sum(vx * vy) / (np.sqrt(np.sum(vx**2)) * np.sqrt(np.sum(vy**2)))
    x_m = np.mean(x)
    y_m = np.mean(y)
    x_s = np.std(x)
    y_s = np.std(y)
    ccc = 2*rho*x_s*y_s/(x_s**2 + y_s**2 + (x_m - y_m)**2)
    return ccc

def metric_for_VA(gt_V,gt_A,pred_V,pred_A):
    ccc_V,ccc_A = CCC_score(gt_V,pred_V),CCC_score(gt_A,pred_A)
    return ccc_V,ccc_A, 0.5*(ccc_V+ccc_A)

def CCC_numpy(y_true, y_pred):
    '''Reference numpy implementation of Lin's Concordance correlation coefficient'''
    
    # covariance between y_true and y_pred
    s_xy = np.cov([y_true, y_pred])[0,1]
    # means
    x_m = np.mean(y_true)
    y_m = np.mean(y_pred)
    # variances
    s_x_sq = np.var(y_true)
    s_y_sq = np.var(y_pred)
    
    # condordance correlation coefficient
    ccc = (2.0*s_xy) / (s_x_sq + s_y_sq + (x_m-y_m)**2)
    
    return ccc

def CCC(y_true, y_pred):
    '''Lin's Concordance correlation coefficient: https://en.wikipedia.org/wiki/Concordance_correlation_coefficient
    
    The concordance correlation coefficient is the correlation between two variables that fall on the 45 degree line through the origin.
    
    It is a product of
    - precision (Pearson correlation coefficient) and
    - accuracy (closeness to 45 degree line)

    Interpretation:
    - `rho_c =  1` : perfect agreement
    - `rho_c =  0` : no agreement
    - `rho_c = -1` : perfect disagreement 
    
    Args: 
    - y_true: ground truth
    - y_pred: predicted values
    
    Returns:
    - concordance correlation coefficient (float)
    '''
    
    
    # covariance between y_true and y_pred
    #N = K.int_shape(y_pred)[-1]
    #s_xy = 1.0 / (N - 1.0 + K.epsilon()) * K.sum((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred)))
    #s_xy = K.mean(K.sum((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred))))
    s_xy = K.mean((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred)))
    # means
    x_m = K.mean(y_true)
    y_m = K.mean(y_pred)
    # variances
    s_x_sq = K.var(y_true)
    s_y_sq = K.var(y_pred)
    
    # condordance correlation coefficient
    ccc = (2.0*s_xy) / (s_x_sq + s_y_sq + (x_m-y_m)**2+K.epsilon())
    #print(s_xy,s_x_sq,s_y_sq,x_m,y_m)
    return ccc

def CCC_VA(y_true, y_pred):
    return 1-0.5*(CCC(y_true[:,0], y_pred[:,0])+CCC(y_true[:,1], y_pred[:,1]))


In [13]:
import math
def f1_score_max_for_AU_one_class(gt, pred, thresh,type=0):
    gt = gt[:,type]
    pred = pred[:,type]
    P = []
    R = []
    ACC = []
    F1 = []
    for i in thresh:
        new_pred = ((pred >= i) * 1).flatten()
        P.append(precision_score(gt.flatten(), new_pred))
        R.append(recall_score(gt.flatten(), new_pred))
        ACC.append(accuracy_score(gt.flatten(), new_pred))
        F1.append(f1_score(gt.flatten(), new_pred))

    F1_MAX = max(F1)
    if F1_MAX < 0 or math.isnan(F1_MAX):
        F1_MAX = 0
        F1_THRESH = 0
        accuracy = 0
    else:
        idx_thresh = np.argmax(F1)
        F1_THRESH = thresh[idx_thresh]
        accuracy = ACC[idx_thresh]
    return F1,F1_MAX,F1_THRESH,accuracy

def f1_score_max(gt, pred, thresh,c=12):
    F1_s = []
    F1_t = []
    ACC = []
    from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
    for i in range(c):
        F1, F1_MAX, F1_THRESH,acc = f1_score_max_for_AU_one_class(gt,pred,thresh,i)
        F1_s.append(F1_MAX)
        F1_t.append(F1_THRESH)
        ACC.append(acc)
    F1_s=np.array(F1_s)
    F1_t=np.array(F1_t)
    ACC=np.array(ACC)
    return F1_s.mean(),F1_t.mean(),F1_s,F1_t,ACC

## Expr 

In [27]:
def get_image2Expr(dirname):
    dirpath=os.path.join(DATA_DIR,'Third ABAW Annotations/EXPR_Classification_Challenge/',dirname)
    num_missed=0
    X,y=[],[]
    for filename in os.listdir(dirpath):
        fn, ext = os.path.splitext(os.path.basename(filename))
        if ext.lower()=='.txt':
            with open(os.path.join(dirpath,filename)) as f:
                lines = f.read().splitlines()
                for i,line in enumerate(lines):
                    if i>0:
                        expression=int(line)
                        if expression>=0:
                            imagename=fn+'/'+get_names(i)+'.jpg'
                            if imagename in filename2featuresAll:
                                X.append(filename2featuresAll[imagename][0])
                                y.append(expression)
                            else:
                                num_missed+=1
    X=np.array(X)
    y=np.array(y)
    print(X.shape,y.shape,num_missed)
    return X,y

X_train,y_train=get_image2Expr('Train_Set')
X_val,y_val=get_image2Expr('Validation_Set')

(585317, 1280) (585317,) 12044
(280532, 1280) (280532,) 3698


In [15]:
def print_expr():
    y_val_preds=mlpModel.predict(X_val)
    y_pred=np.argmax(y_val_preds,axis=1)
    print('Acc:',(y_pred==y_val).mean(), 'F1:',f1_score(y_true=y_val,y_pred=y_pred, average="macro"))
    print(metric_for_Exp(y_val,y_pred))

In [32]:
batch_size=256 #128
mlpModel=Sequential()
if False:
    mlpModel.add(Dense(num_classes, input_shape=X_train.shape[1:],activation='softmax',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size)))
else:
    mlpModel.add(Dense(128, input_shape=X_train.shape[1:],activation='relu')) #256
    mlpModel.add(Dense(num_classes,activation='softmax'))

In [29]:
(unique, counts) = np.unique(y_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[177198  16573  10771   9080  95463  78751  31615 165866] {0: 1.0, 1: 10.691968865021419, 2: 16.451397270448425, 3: 19.515198237885464, 4: 1.856195594104522, 5: 2.2501047605744686, 6: 5.6048711054879, 7: 1.0683202102902343} 8 [0 1 2 3 4 5 6 7]


In [33]:
mlpModel.compile(optimizer=Adam(lr=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlpModel.summary()

save_best_model = SaveBestModel('val_accuracy',True)
mlpModel.fit(X_train,y_train, batch_size=batch_size, epochs=10, verbose=1, 
             callbacks=[save_best_model], validation_data=(X_val,y_val),class_weight=class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               163968    
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 1032      
Total params: 165,000
Trainable params: 165,000
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2287/2287 [==============================] - 12s 5ms/step - loss: 1.5437 - accuracy: 0.7573 - val_loss: 1.9838 - val_accuracy: 0.4660
Epoch 2/10
2287/2287 [==============================] - 11s 5ms/step - loss: 0.5894 - accuracy: 0.8976 - val_loss: 2.0917 - val_accuracy: 0.5003
Epoch 3/10
2287/2287 [==============================] - 11s 5ms/step - loss: 0.4235 - accuracy: 0.9225 - val_loss: 2.2836 - val_accuracy: 0.4972
Epoch 4/10
2287/2287 [==============================] - 11s 5ms/step - loss

In [34]:
if True:
    print_expr()
    print('Best weights:')
    mlpModel.set_weights(best_model_weights)
print_expr()

Acc: 0.4383457145708867 F1: 0.33920330611636373
(0.33920330611636373, 0.4383457145708867, [0.5438041551067051, 0.12640901771336555, 0.5226523887973641, 0.018429057076400072, 0.4351345007276654, 0.4133538326378212, 0.23134583683244686, 0.4224976600391412])
Best weights:
Acc: 0.500285172458044 F1: 0.3807067519117502
(0.3807067519117502, 0.500285172458044, [0.608643679809698, 0.15128569174948728, 0.5162256615077384, 0.01599360255897641, 0.4778312968664514, 0.4611138100075078, 0.3029224246739377, 0.5116378481202044])


In [36]:
if False:
    #mlpModel.save_weights('expr_enet0_7.h5') #Acc: 0.48438324326636534 F1: 0.3604978104476117
    #mlpModel.save_weights('expr_enet0_vgaf.h5')#Acc: 0.49085665806396417 F1: 0.37334516257107847
    mlpModel.save_weights('expr_enet0_vgaf.h5')#Acc: 0.500285172458044 F1: 0.3807067519117502
else:
    mlpModel.load_weights('expr_enet0_vgaf.h5')
    print_expr()

Acc: 0.500285172458044 F1: 0.3807067519117502
(0.3807067519117502, 0.500285172458044, [0.608643679809698, 0.15128569174948728, 0.5162256615077384, 0.01599360255897641, 0.4778312968664514, 0.4611138100075078, 0.3029224246739377, 0.5116378481202044])


### Save test results 

In [37]:
with open(os.path.join(DATA_DIR,'Expression_Classification_Challenge_test_set_release.txt'),'r') as f:
    test_set_videos=f.read().splitlines()
print(len(test_set_videos),test_set_videos[:5])

228 ['14-30-1920x1080', '16-30-1920x1080', '40-30-1280x720', '43-30-406x720', '44-25-426x240']


In [38]:
data_dir=os.path.join(DATA_DIR,'cropped')
test_videos={}
for videoname in os.listdir(data_dir):
    #if videoname not in train_val_videos:
    if videoname in test_set_videos:
        X,indices,filenames=[],[],[]
        images=[img_name for img_name in os.listdir(os.path.join(data_dir,videoname)) if img_name.lower().endswith('.jpg')]
        for img_name in sorted(images, key=compare_filenames):
            k=videoname+'/'+img_name
            if k in filename2featuresAll:
                X.append(filename2featuresAll[k][0])
                indices.append(int(img_name[:-4]))
                filenames.append(k)
        test_videos[videoname]=(mlpModel.predict(np.array(X)),indices,filenames)
print(len(test_videos))

228


In [41]:
delta=2 # 0 2 4 7
resdir=os.path.join(DATA_DIR,'test_results/2_smooth5/EXPR')
if not os.path.exists(resdir):
    os.makedirs(resdir)
header = 'Neutral,Anger,Disgust,Fear,Happiness,Sadness,Surprise,Other\n'
for videoname,(y_pred_expr,indices,filenames) in tqdm(test_videos.items()):
    #y_pred_expr=mlpModel.predict(X)
    #print(len(indices),indices[-1],indices[0])
    #print(y_pred_expr.shape)
    
    cur_ind=0
    preds_proba=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds_proba.append(y_pred_expr[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds_proba.append(y_pred_expr[cur_ind])
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                preds_proba.append(pred)
    
    pred=y_pred_expr[cur_ind-1]
    if videoname.endswith('_left'):
        videoname=videoname[:-5]
    elif videoname.endswith('_right'):
        videoname=videoname[:-6]
    for _ in range(indices[-1],video2len[videoname]):
        preds_proba.append(pred)

    preds_proba=np.array(preds_proba)
    with open(os.path.join(resdir,videoname+'.txt'), 'w') as f:
        f.write(header)
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            proba=preds_proba[i1:i+delta+1].mean(axis=0)
            f.write(str(np.argmax(proba))+'\n')

100%|██████████| 228/228 [00:49<00:00,  4.57it/s]


## Valence-Arousal 

In [42]:
def get_image2VA(dirname):
    dirpath=os.path.join(DATA_DIR,'Third ABAW Annotations/VA_Estimation_Challenge/',dirname)
    num_missed=0
    X,y=[],[]
    for filename in os.listdir(dirpath):
        fn, ext = os.path.splitext(os.path.basename(filename))
        if ext.lower()=='.txt':
            with open(os.path.join(dirpath,filename)) as f:
                lines = f.read().splitlines()
                for i,line in enumerate(lines):
                    if i>0:
                        splitted_line=line.split(',')
                        valence=float(splitted_line[0])
                        arousal=float(splitted_line[1])
                        if valence>=-1 and arousal>=-1:
                            imagename=fn+'/'+get_names(i)+'.jpg'
                            if imagename in filename2featuresAll:
                                X.append(filename2featuresAll[imagename][1])
                                y.append((valence,arousal))
                            else:
                                num_missed+=1
    X=np.array(X)
    y=np.array(y)
    print(X.shape,y.shape,num_missed)
    return X,y

X_train,y_train=get_image2VA('Train_Set')
X_val,y_val=get_image2VA('Validation_Set')

(1555919, 8) (1555919, 2) 25265
(338755, 8) (338755, 2) 5959


In [43]:
def print_ccc():
    y_val_preds=mlpModel.predict(X_val)
    gt_V=y_val[:,0]
    gt_A=y_val[:,1]
    pred_V=y_val_preds[:,0]
    pred_A=y_val_preds[:,1]
    print(metric_for_VA(gt_V,gt_A,pred_V,pred_A))
    print(CCC_numpy(gt_V,pred_V),CCC_numpy(gt_A,pred_A))

In [44]:
batch_size=512 #128
mlpModel=Sequential()
if True:
    mlpModel.add(Dense(2, input_shape=X_train.shape[1:],activation='tanh',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size)))
else:
    mlpModel.add(Dense(128, input_shape=X_train.shape[1:],activation='relu')) #256
    mlpModel.add(Dense(2,activation='tanh'))

In [45]:
mlpModel.compile(optimizer=Adam(lr=1e-3), loss=CCC_VA, metrics=['mae','mse'])
mlpModel.summary()

save_best_model = SaveBestModel('val_loss',False)
mlpModel.fit(X_train,y_train, batch_size=batch_size, epochs=20, verbose=1, callbacks=[save_best_model], validation_data=(X_val,y_val))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 2)                 18        
Total params: 18
Trainable params: 18
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
3039/3039 [==============================] - 15s 5ms/step - loss: 0.8378 - mae: 0.6590 - mse: 0.6505 - val_loss: 0.8218 - val_mae: 0.2785 - val_mse: 0.1305
Epoch 2/20
3039/3039 [==============================] - 15s 5ms/step - loss: 0.5054 - mae: 0.2777 - mse: 0.1278 - val_loss: 0.8217 - val_mae: 0.2772 - val_mse: 0.1292
Epoch 3/20
3039/3039 [==============================] - 15s 5ms/step - loss: 0.5061 - mae: 0.2777 - mse: 0.1278 - val_loss: 0.8217 - val_mae: 0.2785 - val_mse: 0.1307
Epoch 4/20
3039/3039 [==============================] - 15s 5ms/step - loss: 0.5057 - mae: 0.2774 - mse: 0.1276 - val_loss: 0.8232 - val_mae: 0.2802 - 

In [46]:
print_ccc()
print('Best weights:')
mlpModel.set_weights(best_model_weights)
print_ccc()

(0.4315120893088298, 0.4936346386620984, 0.4625733639854641)
0.43151336571687304 0.49363607254835185
Best weights:
(0.4292276337077572, 0.4967911728286142, 0.4630094032681857)
0.4292288924922802 0.49679262232452065


In [38]:
if False:
    #mlpModel.save_weights('va_enet0_7.h5') #(0.42432524480841777, 0.4820651093839277, 0.45319517709617274)
    mlpModel.save_weights('va_enet0_vgaf.h5') #(0.42896714840390443, 0.4963675256226998, 0.4626673370133021)
else:
    mlpModel.load_weights('va_enet0_vgaf.h5')
    print_ccc()

(0.42896714840390443, 0.4963675256226998, 0.4626673370133021)
0.4289684062520081 0.4963689931092825


### Save test results 

In [39]:
with open(os.path.join(DATA_DIR,'Valence_Arousal_Estimation_Challenge_test_set_release.txt'),'r') as f:
    test_set_videos=f.read().splitlines()
print(len(test_set_videos),test_set_videos[:5])

152 ['2-30-640x360', '3-25-1920x1080', '6-30-1920x1080_left', '6-30-1920x1080_right', '11-24-1920x1080']


In [40]:
data_dir=os.path.join(DATA_DIR,'cropped')
test_videos={}
for videoname in os.listdir(data_dir):
    #if videoname not in train_val_videos:
    if videoname in test_set_videos:
        X,indices=[],[]
        images=[img_name for img_name in os.listdir(os.path.join(data_dir,videoname)) if img_name.lower().endswith('.jpg')]
        for img_name in sorted(images, key=compare_filenames):
            k=videoname+'/'+img_name
            if k in filename2featuresAll:
                X.append(filename2featuresAll[k][1])
                indices.append(int(img_name[:-4]))
        test_videos[videoname]=(mlpModel.predict(np.array(X)),indices)
print(len(test_videos))

152


In [42]:
delta=2 #0 2 4 7
resdir=os.path.join(DATA_DIR,'test_results/2_smooth5/VA')
if not os.path.exists(resdir):
    os.makedirs(resdir)
header = 'valence,arousal\n'

for videoname,(y_pred_va,indices) in test_videos.items():
    #y_pred_va=mlpModel.predict(X)
    #print(len(indices),indices[-1],indices[0])
    #print(y_pred_va.shape)
    cur_ind=0
    preds=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds.append(y_pred_va[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds.append(y_pred_va[cur_ind])
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_va[cur_ind-1]+(1-w)*y_pred_va[cur_ind]
                preds.append(pred)

    pred=y_pred_va[cur_ind-1]
    if videoname.endswith('_left'):
        videoname=videoname[:-5]
    elif videoname.endswith('_right'):
        videoname=videoname[:-6]
    for _ in range(indices[-1],video2len[videoname]):
        preds.append(pred)
    preds=np.array(preds)
    with open(os.path.join(resdir,videoname+'.txt'), 'w') as f:
        f.write(header)
        for i in range(len(preds)):
            i1=max(i-delta,0)
            pred=preds[i1:i+delta+1].mean(axis=0)
            f.write(str(pred[0])+','+str(pred[1])+'\n')

100%|██████████| 152/152 [00:23<00:00,  6.45it/s]


## AU 

In [47]:
def get_image2AU(dirname):
    dirpath=os.path.join(DATA_DIR,'Third ABAW Annotations/AU_Detection_Challenge/',dirname)
    num_missed=0
    X,y=[],[]
    for filename in os.listdir(dirpath):
        fn, ext = os.path.splitext(os.path.basename(filename))
        if ext.lower()=='.txt':
            with open(os.path.join(dirpath,filename)) as f:
                lines = f.read().splitlines()
                for i,line in enumerate(lines):
                    if i>0:
                        splitted_line=line.split(',')
                        aus=list(map(int,splitted_line))
                        if min(aus)>=0:
                            imagename=fn+'/'+get_names(i)+'.jpg'
                            if imagename in filename2featuresAll:
                                X.append(filename2featuresAll[imagename][0])
                                y.append(aus)
                            else:
                                num_missed+=1
    X=np.array(X)
    y=np.array(y)
    print(X.shape,y.shape,num_missed)
    return X,y

X_train,y_train=get_image2AU('Train_Set')
X_val,y_val=get_image2AU('Validation_Set')

(1356861, 1280) (1356861, 12) 2829
(445836, 1280) (445836, 12) 9


In [48]:
def print_au():
    y_val_preds=mlpModel.predict(X_val)
    new_pred = ((y_val_preds >= 0.5) * 1)
    print(np.mean([f1_score(y_true=y_val[:,i],y_pred=new_pred[:,i]) for i in range(y_val_preds.shape[1])]))
    print(f1_score_max(y_val,y_val_preds,thresh=np.arange(0.1,1,0.1)))


In [49]:
num_labels=y_train.shape[1]
print(num_labels)
from sklearn.utils.class_weight import compute_class_weight
class_weights = np.empty([num_labels, 2])
for i in range(num_labels):
    neg, pos = np.bincount(y_train[:, i])
    total = neg + pos
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weights[i][0]=weight_for_0
    class_weights[i][1]=weight_for_1
    #class_weights[i] = compute_class_weight('balanced', [0,1], y_train[:, i])
print(class_weights)

12
[[ 0.56699737  4.23148962]
 [ 0.52623698 10.02853659]
 [ 0.59099307  3.24746184]
 [ 0.68134051  1.87862194]
 [ 0.82949071  1.25874673]
 [ 0.7628283   1.45119134]
 [ 0.66223812  2.04094492]
 [ 0.51446878 17.77857704]
 [ 0.51624061 15.8935131 ]
 [ 0.51314264 19.52205628]
 [ 1.3391547   0.79791885]
 [ 0.54019984  6.71892981]]


In [50]:
if False:
    loss='binary_crossentropy'
    #loss='hinge'
else:
    import tensorflow.keras.backend as K
    def get_weighted_loss(weights):
        def weighted_loss(y_true, y_pred):
            y_true=tf.cast(y_true, tf.float32)
            ce=K.binary_crossentropy(y_true, y_pred)
            return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*ce, axis=-1)
        return weighted_loss
    loss=get_weighted_loss(class_weights)

In [51]:
metrics=[tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()] # 

In [52]:
batch_size=512 #128
mlpModel=Sequential()
if False:
    mlpModel.add(Dense(y_train.shape[1], input_shape=X_train.shape[1:],activation='sigmoid',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size)))
else:
    mlpModel.add(Dense(128, input_shape=X_train.shape[1:],activation='relu')) #256
    mlpModel.add(Dense(y_train.shape[1],activation='sigmoid'))

In [53]:
mlpModel.compile(optimizer=Adam(lr=1e-3), loss=loss, metrics=metrics)
mlpModel.summary()

save_best_model = SaveBestModel('val_loss',False)
mlpModel.fit(X_train,y_train, batch_size=batch_size, epochs=20, verbose=1, callbacks=[save_best_model], validation_data=(X_val,y_val))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               163968    
_________________________________________________________________
dense_10 (Dense)             (None, 12)                1548      
Total params: 165,516
Trainable params: 165,516
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
2651/2651 [==============================] - 24s 9ms/step - loss: 0.3646 - auc: 0.9142 - binary_accuracy: 0.8373 - recall: 0.8112 - precision: 0.5625 - val_loss: 0.6748 - val_auc: 0.8232 - val_binary_accuracy: 0.8127 - val_recall: 0.7541 - val_precision: 0.5328
Epoch 2/20
2651/2651 [==============================] - 22s 8ms/step - loss: 0.2625 - auc: 0.9553 - binary_accuracy: 0.8848 - recall: 0.8638 - precision: 0.6578 - val_loss: 0.7679 - val_auc: 0.8200 - val_binary_accuracy: 0.8261 - val_rec

In [54]:
print_au()
print('Best weights:')
mlpModel.set_weights(best_model_weights)
print_au()

0.4942478028701414
(0.5013035654863197, 0.5916666666666667, array([0.51604332, 0.44447088, 0.52404717, 0.58320476, 0.70805253,
       0.72693301, 0.6950011 , 0.32262834, 0.15273545, 0.16133106,
       0.82544436, 0.35575082]), array([0.7, 0.7, 0.7, 0.4, 0.5, 0.5, 0.6, 0.8, 0.6, 0.9, 0.2, 0.5]), array([0.86599781, 0.90572094, 0.84490934, 0.77399986, 0.74561498,
       0.79577917, 0.84695942, 0.94862236, 0.92392718, 0.94934909,
       0.75133233, 0.83620659]))
Best weights:
0.49556686087276947
(0.5291290387995752, 0.6749999999999999, array([0.5589234 , 0.46158495, 0.5535661 , 0.61873531, 0.73861687,
       0.7435496 , 0.72794563, 0.34284344, 0.19011839, 0.21411149,
       0.8340475 , 0.36550579]), array([0.7, 0.8, 0.7, 0.6, 0.5, 0.6, 0.7, 0.9, 0.8, 0.8, 0.3, 0.7]), array([0.87326057, 0.90807831, 0.86248082, 0.80650284, 0.77540396,
       0.80970133, 0.86512978, 0.950816  , 0.9424609 , 0.94001606,
       0.76927839, 0.84234562]))


In [46]:
if False:
    #mlpModel.save_weights('au_enet0_7.h5') #0.4992152981157956   with thresholds 0.5228919921944153/0.5228095834398018
    mlpModel.save_weights('au_enet0_vgaf.h5') #0.507568027779308
else:
    mlpModel.load_weights('au_enet0_vgaf.h5')
    print_au()

0.507568027779308
(0.5367145284583944, 0.65, array([0.56438654, 0.47163507, 0.57207889, 0.61677155, 0.74818918,
       0.74534595, 0.71942028, 0.35743029, 0.19360184, 0.24140214,
       0.83680711, 0.37350551]), array([0.8, 0.8, 0.7, 0.5, 0.5, 0.5, 0.6, 0.8, 0.8, 0.8, 0.3, 0.7]), array([0.8884859 , 0.90530374, 0.8692748 , 0.79757579, 0.78782557,
       0.80682583, 0.8569631 , 0.94949937, 0.94323473, 0.95116814,
       0.77594452, 0.85026333]))


In [47]:
pred=mlpModel.predict(X_val)
#thresholds=0.5
#thresholds=np.array([0.7, 0.8, 0.7, 0.6, 0.5, 0.5, 0.6, 0.8, 0.7, 0.8, 0.3, 0.7]) #au_enet0_7 - 0.5228095834398018
thresholds=np.array([0.8, 0.8, 0.7, 0.5, 0.5, 0.5, 0.6, 0.8, 0.8, 0.8, 0.3, 0.7]) #vgaf 0.5367145284583944
new_pred = ((pred >= thresholds) * 1)
print(np.mean([f1_score(y_true=y_val[:,i],y_pred=new_pred[:,i]) for i in range(pred.shape[1])]))

0.5367145284583944


### Save test results

In [48]:
with open(os.path.join(DATA_DIR,'Action_Unit_Detection_Challenge_test_set_release.txt'),'r') as f:
    test_set_videos=f.read().splitlines()
print(len(test_set_videos),test_set_videos[:5])

141 ['2-30-640x360', '3-25-1920x1080', '6-30-1920x1080_left', '6-30-1920x1080_right', '11-24-1920x1080']


In [49]:
data_dir=os.path.join(DATA_DIR,'cropped')
test_videos={}
for videoname in os.listdir(data_dir):
    #if videoname not in train_val_videos:
    if videoname in test_set_videos:
        X,indices=[],[]
        images=[img_name for img_name in os.listdir(os.path.join(data_dir,videoname)) if img_name.lower().endswith('.jpg')]
        for img_name in sorted(images, key=compare_filenames):
            k=videoname+'/'+img_name
            if k in filename2featuresAll:
                X.append(filename2featuresAll[k][0])
                indices.append(int(img_name[:-4]))
        test_videos[videoname]=(mlpModel.predict(np.array(X)),indices)
print(len(test_videos))

141


In [50]:
delta=7 #0 2 4 7
resdir=os.path.join(DATA_DIR,'test_results/4_smooth15/AU')
if not os.path.exists(resdir):
    os.makedirs(resdir)
header = 'AU1,AU2,AU4,AU6,AU7,AU10,AU12,AU15,AU23,AU24,AU25,AU26\n'

for videoname,(y_pred_aus,indices) in tqdm(test_videos.items()):
    #if videoname!='video32':
    #    continue
    #y_pred_aus=mlpModel.predict(X)
    #print(len(indices),indices[-1],indices[0])
    #print(y_pred_aus.shape)
    cur_ind=0
    preds=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds.append(y_pred_aus[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds.append(y_pred_aus[cur_ind])
            
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_aus[cur_ind-1]+(1-w)*y_pred_aus[cur_ind]
                preds.append(pred)
                
    pred=y_pred_aus[cur_ind-1]
    if videoname.endswith('_left'):
        videoname=videoname[:-5]
    elif videoname.endswith('_right'):
        videoname=videoname[:-6]
    for _ in range(indices[-1],video2len[videoname]):
        preds.append(pred)
    
    preds=np.array(preds)
    with open(os.path.join(resdir,videoname+'.txt'), 'w') as f:
        f.write(header)
        for i in range(len(preds)):
            i1=max(i-delta,0)
            pred=preds[i1:i+delta+1].mean(axis=0)
            aus=(pred>=thresholds)*1
            f.write(','.join(map(str,aus))+'\n')

100%|██████████| 141/141 [00:37<00:00,  3.72it/s]


## Multi-task challenge 

In [27]:
def get_image2all(filename):
    with open(os.path.join(DATA_DIR,'Third ABAW Annotations/MTL_Challenge/'+filename)) as f:
        mtl_lines = f.read().splitlines()
    num_missed=0
    X,y_va,y_expr,y_aus=[],[],[],[]
    masks_va,masks_expr,masks_aus=[],[],[]
    for line in mtl_lines[1:]:
        splitted_line=line.split(',')
        imagename=splitted_line[0]
        valence=float(splitted_line[1])
        arousal=float(splitted_line[2])
        expression=int(splitted_line[3])
        aus=list(map(int,splitted_line[4:]))
        
        mask_VA=(valence>-5 and arousal>-5)
        if not mask_VA:
            valence=arousal=0
            
        mask_expr=(expression>-1)
        if not mask_expr:
            expression=0
            
        mask_aus=min(aus)>=0
        if not mask_aus:
            aus=[0]*len(aus)
        #print(imagename,valence,arousal,expression,aus)
        if mask_VA or mask_expr or mask_aus:
            if imagename in filename2featuresAll:
                #X.append(filename2featuresAll[imagename][0])
                X.append(np.concatenate((filename2featuresAll[imagename][0],filename2featuresAll[imagename][1])))
                y_va.append((valence,arousal))
                masks_va.append(mask_VA)
                
                y_expr.append(expression)
                masks_expr.append(mask_expr)
                
                y_aus.append(aus)
                masks_aus.append(mask_aus)
            else:
                num_missed+=1
        #elif (valence==-5 or arousal==-5) and valence!=arousal:
        #    print(valence,arousal)
    X=np.array(X)
    y_va=np.array(y_va)
    y_expr=np.array(y_expr)
    y_aus=np.array(y_aus)
    masks_va=np.array(masks_va).astype(np.float32)
    masks_expr=np.array(masks_expr).astype(np.float32)
    masks_aus=np.array(masks_aus).astype(np.float32)
    print(X.shape,y_va.shape,y_expr.shape,y_aus.shape,masks_va.shape,num_missed)
    return X,y_va,y_expr,y_aus,masks_va,masks_expr,masks_aus

X_train,y_va_train,y_expr_train,y_aus_train,masks_va_train,masks_expr_train,masks_aus_train=get_image2all('train_set.txt')
X_val,y_va_val,y_expr_val,y_aus_val,masks_va_val,masks_expr_val,masks_aus_val=get_image2all('validation_set.txt')

(142225, 1416) (142225, 2) (142225,) (142225, 12) (142225,) 2666
(26876, 1416) (26876, 2) (26876,) (26876, 12) (26876,) 211


In [23]:
(unique, counts) = np.unique(y_expr_train[masks_expr_train==1.].astype(int), return_counts=True)
num_classes=len(unique)
emo_cw=1/counts
emo_cw/=emo_cw.min()
emo_class_weights = {i:cwi for i,cwi in zip(unique,emo_cw)}
print(counts, emo_class_weights, num_classes, unique)
print(emo_cw)

[23958  4555  3292  3122 18141  7482  5258 24852] {0: 1.0373153017781116, 1: 5.455982436882547, 2: 7.54921020656136, 3: 7.960281870595772, 4: 1.3699355052091946, 5: 3.321571772253408, 6: 4.7265119817421075, 7: 1.0} 8 [0 1 2 3 4 5 6 7]
[1.0373153  5.45598244 7.54921021 7.96028187 1.36993551 3.32157177
 4.72651198 1.        ]


In [24]:
num_labels=y_aus_train.shape[1]
print(num_labels)
from sklearn.utils.class_weight import compute_class_weight
aus_class_weights = np.empty([num_labels, 2])
for i in range(num_labels):
    neg, pos = np.bincount(y_aus_train[masks_aus_train==1., i])
    total = neg + pos
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    aus_class_weights[i][0]=weight_for_0
    aus_class_weights[i][1]=weight_for_1
    #aus_class_weights[i] = compute_class_weight('balanced', [0,1], y_train[:, i])
print(aus_class_weights)

12
[[ 0.61103657  2.75151045]
 [ 0.54642804  5.88467696]
 [ 0.62178604  2.5527804 ]
 [ 0.70912912  1.69543373]
 [ 0.83358711  1.24942945]
 [ 0.79504233  1.34733604]
 [ 0.67451569  1.93253596]
 [ 0.51219743 20.99612403]
 [ 0.51475913 17.43866486]
 [ 0.52554636 10.28612832]
 [ 1.57519131  0.73251676]
 [ 0.55806602  4.80544402]]


In [25]:
def loss_va(y_true, y_pred):
    res=1-0.5*(CCC(y_true[:,0], y_pred[:,0])+CCC(y_true[:,1], y_pred[:,1]))
    #res=K.mean((y_true[:,0] - y_pred[:,0]) * (y_true[:,0] - y_pred[:,0]))+K.mean((y_true[:,1] - y_pred[:,1]) * (y_true[:,1] - y_pred[:,1]))
    #print(res)
    return res

In [26]:
class WeightedExprSCCE(tf.keras.losses.Loss):
    def __init__(self, class_weight, from_logits=False, name='expr_scce'):
        if class_weight is None or all(v == 1. for v in class_weight):
            self.class_weight = None
        else:
            self.class_weight = tf.convert_to_tensor(class_weight,
                dtype=tf.float32)
        self.reduction = tf.keras.losses.Reduction.NONE
        self.unreduced_scce = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=from_logits, name=name,
            reduction=self.reduction)

    def __call__(self, y_true, y_pred, sample_weight=None):
        loss = self.unreduced_scce(y_true, y_pred, sample_weight)
        if self.class_weight is not None:
            weight_mask = tf.gather(self.class_weight, y_true)
            loss = tf.math.multiply(loss, weight_mask)
        loss=K.mean(loss)
        return loss
loss_expr=WeightedExprSCCE(emo_cw)

In [27]:
def get_weighted_loss_aus(weights):
    def weighted_loss(y_true, y_pred):
        y_true=tf.cast(y_true, tf.float32)
        ce=K.binary_crossentropy(y_true, y_pred)
        res=K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*ce)#, axis=-1)
        #print(res)
        return res
    return weighted_loss
loss_aus=get_weighted_loss_aus(aus_class_weights)

In [28]:
metrics=[tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()] # 

In [28]:
batch_size=256 #128
img = tf.keras.Input(shape=X_train.shape[1:])
mask1 = tf.keras.Input(shape=(1,))
mask2 = tf.keras.Input(shape=(1,))
mask3 = tf.keras.Input(shape=(1,))
x=img
#x=Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
va_out = Dense(2, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
va_out_masked=tf.keras.layers.Multiply(name='va_out')([va_out,mask1])

#x=va_out

expr_out=Dense(8, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
expr_out_masked=tf.keras.layers.Multiply(name='expr_out')([expr_out,mask2])
aus_out=Dense(12, activation='sigmoid',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
aus_out_masked=tf.keras.layers.Multiply(name='aus_out')([aus_out,mask3])

mtlModel=tf.keras.Model(inputs=[img,mask1,mask2,mask3], outputs=[va_out_masked, expr_out_masked,aus_out_masked])

In [46]:
mtlModel.compile(optimizer=Adam(lr=1e-3), loss=[loss_va,loss_expr,loss_aus], metrics=[["mean_absolute_error"], ["accuracy"],metrics])
mtlModel.summary()

save_best_model = SaveBestModel('val_loss',False)
mtlModel.fit([X_train,masks_va_train,masks_expr_train,masks_aus_train],
             [y_va_train,y_expr_train,y_aus_train], batch_size=batch_size, epochs=20, 
             verbose=1, callbacks=[save_best_model], 
             validation_data=([X_val,masks_va_val,masks_expr_val,masks_aus_val],[y_va_val,y_expr_val,y_aus_val]))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 1408)]       0                                            
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 128)          180352      input_13[0][0]                   
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 2)            258         dense_10[0][0]                   
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 1)]          0                                            
____________________________________________________________________________________________

556/556 [==============================] - 7s 12ms/step - loss: 3.1504 - va_out_loss: 0.2458 - expr_out_loss: 2.1705 - aus_out_loss: 0.3520 - va_out_mean_absolute_error: 0.1503 - expr_out_accuracy: 0.8510 - aus_out_aus_out_aus_out_aus_out_auc: 0.9057 - aus_out_aus_out_aus_out_aus_out_binary_accuracy: 0.8472 - aus_out_aus_out_aus_out_aus_out_recall: 0.7388 - aus_out_aus_out_aus_out_aus_out_precision: 0.5178 - val_loss: 5.8400 - val_va_out_loss: 0.7938 - val_expr_out_loss: 4.0787 - val_aus_out_loss: 0.5843 - val_va_out_mean_absolute_error: 0.2915 - val_expr_out_accuracy: 0.6219 - val_aus_out_aus_out_aus_out_aus_out_auc: 0.7628 - val_aus_out_aus_out_aus_out_aus_out_binary_accuracy: 0.7447 - val_aus_out_aus_out_aus_out_aus_out_recall: 0.6654 - val_aus_out_aus_out_aus_out_aus_out_precision: 0.4939
Epoch 18/20
556/556 [==============================] - 7s 12ms/step - loss: 3.1657 - va_out_loss: 0.2482 - expr_out_loss: 2.1798 - aus_out_loss: 0.3532 - va_out_mean_absolute_error: 0.1511 - expr_

In [29]:
def print_all():
    y_pred_va,y_pred_expr,y_pred_aus=mtlModel.predict([X_val,masks_va_val,masks_expr_val,masks_aus_val])
    print('\nAV')
    gt_V=y_va_val[masks_va_val==1,0]
    gt_A=y_va_val[masks_va_val==1,1]
    pred_V=y_pred_va[masks_va_val==1,0]
    pred_A=y_pred_va[masks_va_val==1,1]
    ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
    print(gt_V.shape,ccc_V,ccc_A,ccc_VA)
    
    print('\nExpression')
    print(y_expr_val[masks_expr_val==1].shape)
    y_pred=np.argmax(y_pred_expr,axis=1)
    print((y_pred==y_expr_val)[masks_expr_val==1].mean())
    f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
    print(f1_expr)
    print(metric_for_Exp(y_expr_val[masks_expr_val==1],y_pred[masks_expr_val==1]))
    
    print('\nAUs')
    new_pred = ((y_pred_aus >= 0.5) * 1)
    print(new_pred[masks_aus_val==1,:].shape)
    f1_au=np.mean([f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])])
    print(f1_au)
    print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))
    
    total=ccc_VA+f1_expr+f1_au
    print('\nTotal',ccc_VA,f1_expr,f1_au,total)

In [47]:
print_all()


AV
(26876,) 0.4090471734064517 0.3674063091893927 0.38822674129792223

Expression
(15440,)
0.34682642487046633
0.26504026317339263
(0.26504026317339263, 0.34682642487046633, [0.3456547384978372, 0.32295719844357973, 0.2906474820143885, 0.07128446536650976, 0.3909541511771995, 0.21816601477618427, 0.05974607916355489, 0.4209119759478871])

AUs
(26876, 12)
0.46025544413804814


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4859314996097397, 0.5249999999999999, array([0.51618705, 0.42366046, 0.53611987, 0.54167252, 0.73734781,
       0.71091839, 0.64928825, 0.1347452 , 0.10688909, 0.24895397,
       0.87449997, 0.35089541]), array([0.6, 0.7, 0.5, 0.4, 0.4, 0.4, 0.5, 0.5, 0.6, 0.8, 0.3, 0.6]), array([0.7998214 , 0.88833904, 0.78114303, 0.63599494, 0.7030064 ,
       0.70977824, 0.78274297, 0.75677184, 0.89243191, 0.94656943,
       0.79338443, 0.81388599]))

Total 0.38822674129792223 0.26504026317339263 0.46025544413804814 1.1135224486093631


In [48]:
print('Best weights:')
mtlModel.set_weights(best_model_weights)
print_all()

Best weights:

AV
(26876,) 0.3797703810410656 0.3558372646729406 0.3678038228570031

Expression
(15440,)
0.34177461139896376
0.2800059725435074
(0.2800059725435074, 0.34177461139896376, [0.3581818181818182, 0.3095429029671211, 0.2803468208092486, 0.05013927576601671, 0.3791107974594213, 0.36765772298767224, 0.11022657685241886, 0.3848418653243423])

AUs
(26876, 12)
0.4564833238081867


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.48430574871900606, 0.5416666666666666, array([0.51105217, 0.41157324, 0.52681341, 0.54620102, 0.73731634,
       0.71095461, 0.6485908 , 0.13312813, 0.12139219, 0.2482679 ,
       0.87653807, 0.33984112]), array([0.6, 0.7, 0.5, 0.5, 0.4, 0.5, 0.5, 0.6, 0.6, 0.8, 0.2, 0.6]), array([0.79424021, 0.87892544, 0.76164608, 0.67510046, 0.70929454,
       0.7495535 , 0.78010121, 0.83234112, 0.92297961, 0.95155529,
       0.79055663, 0.81756958]))

Total 0.3678038228570031 0.2800059725435074 0.4564833238081867 1.104293119208697


In [36]:
if False:
    mtlModel.save_weights('mtl_enet2_8.h5') #0.3837151018856935 0.3022424974391059 0.46080879636948063 1.1467663956942802
else:
    mtlModel.load_weights('mtl_enet2_8.h5')
    print_all()


AV
(26876,) 0.3979329061613654 0.36949729761002165 0.3837151018856935

Expression
(15440,)
0.37480569948186526
0.3022424974391059
(0.3022424974391059, 0.37480569948186526, [0.3598076604211573, 0.32841565105837073, 0.25925925925925924, 0.1281094527363184, 0.4816826856108093, 0.30715123094958974, 0.13212121212121214, 0.42139282735613015])

AUs
(26876, 12)
0.46080879636948063


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.49019032775732496, 0.5416666666666666, array([0.5339532 , 0.42508051, 0.57284289, 0.55691307, 0.73366902,
       0.70831596, 0.64739687, 0.11086862, 0.08219178, 0.27545552,
       0.87113181, 0.36446469]), array([0.5, 0.7, 0.5, 0.4, 0.4, 0.5, 0.5, 0.6, 0.6, 0.8, 0.3, 0.7]), array([0.77247358, 0.88707397, 0.79130079, 0.62486977, 0.70479238,
       0.73976782, 0.76942253, 0.77262241, 0.92271915, 0.9496949 ,
       0.78787766, 0.86504688]))

Total 0.3837151018856935 0.3022424974391059 0.46080879636948063 1.1467663956942802


In [37]:
if True:
    y_pred_va,y_pred_expr,y_pred_aus=mtlModel.predict([X_val,masks_va_val,masks_expr_val,masks_aus_val])
thresholds=np.array([0.5, 0.7, 0.5, 0.4, 0.4, 0.5, 0.5, 0.6, 0.6, 0.8, 0.3, 0.7])
new_pred = ((y_pred_aus >= thresholds) * 1)
f1_au=np.mean([f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])])
print(f1_au)


0.49019032775732496


### Save test results 

In [38]:
with open(os.path.join(DATA_DIR,'Multi_Task_Learning_Challenge_test_set_release.txt'),'r') as f:
    test_set_videos=f.read().splitlines()
print(len(test_set_videos),test_set_videos[:5])

51246 ['image,valence,arousal,expression,aus', 'video45_1/00001.jpg', 'video45_1/00008.jpg', 'video45_1/00017.jpg', 'video45_1/00021.jpg']


In [39]:
resdir=os.path.join(DATA_DIR,'test_results/MTL')
header = 'image,valence,arousal,expression,aus\n'
with open(os.path.join(resdir,'MTL_predictions.txt'), 'w') as f:
    f.write(header)
    X,filenames=[],[]
    mask_ones=np.ones(batch_size)
    
    for k in test_set_videos[1:]:
        if k in filename2featuresAll:
            X.append(np.concatenate((filename2featuresAll[k][0],filename2featuresAll[k][1])))
            filenames.append(k)
            
            if len(X)==batch_size:
                y_pred_va,y_pred_expr,y_pred_aus=mtlModel.predict([np.array(X),mask_ones,mask_ones,mask_ones])
                y_expr=np.argmin(y_pred_expr,axis=1)
                y_aus=(y_pred_aus>=thresholds)*1

                for filename, va,expr,aus in zip(filenames,y_pred_va,y_expr,y_aus):
                    s=','.join([filename,str(va[0]),str(va[1]),str(expr),*map(str,aus)])+'\n'
                    f.write(s)
                X,filenames=[],[]
        else:
            print(k, 'not found')
    if len(X)>0:
        mask_ones=np.ones(len(X))
        y_pred_va,y_pred_expr,y_pred_aus=mtlModel.predict([np.array(X),mask_ones,mask_ones,mask_ones])
        y_expr=np.argmin(y_pred_expr,axis=1)
        y_aus=(y_pred_aus>=thresholds)*1

        for filename, va,expr,aus in zip(filenames,y_pred_va,y_expr,y_aus):
            s=','.join([filename,str(va[0]),str(va[1]),str(expr),*map(str,aus)])+'\n'
            f.write(s)
        X,filenames=[],[]